In [10]:
from src.scraping.googlescraping.parallel_browser_manager import ParallelBrowserManager

# Initialize the ParallelBrowserManager with the desired number of workers
parallel_manager = ParallelBrowserManager(num_workers=5)

# List of numbers to search
numbers_to_search = [1, 2, 3, 4, 5]

# Perform the searches in parallel
parallel_manager.perform_search(numbers_to_search)


Search for 5 completed with result: True
Search for 3 completed with result: True
Search for 1 completed with result: True
Search for 2 completed with result: True
Search for 4 completed with result: True
